## This is the notebook for Week 4 of the final capstone project

#### A description of the problem and a discussion of the background:

> <b>Problem Statement:</b> Create an algorithm that can find neighborhoods which are similar to a given neighborhood in New York city. 
 <b>Who will be interested:</b> This can be used by relocation agencies to help people who are looking to move to Toronto from New York.
>> <b>Background:</b> Toronto is a fast growing city in North America. Having said so, there are loads of new opportunities that are sprawling across Toronto that is of interest to people. This has led to a recent growth in immigration to the city. As per this <a href='https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=2&cad=rja&uact=8&ved=2ahUKEwiiiv7ihqbjAhWPtp4KHX-kCA8QFjABegQIDBAF&url=https%3A%2F%2Fwww12.statcan.gc.ca%2Fnhs-enm%2F2011%2Fas-sa%2F99-010-x%2F99-010-x2011001-eng.cfm&usg=AOvVaw2_J8qJuBqdE4dWLNyJOajW'>site</a>; of all immigrants in Ontario, 7 out of 10 lived in Toronto. This has led to many immigration services that cater to the city of Toronto,ON. This has also led to a rich culturally diverse Toronto. Huffington POst also came up with this widely read <a href='https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=2ahUKEwjcsfPMh6bjAhXHjp4KHVPFDAUQFjAAegQIABAB&url=https%3A%2F%2Fwww.huffpost.com%2Fentry%2F9-reasons-to-leave-new-york-for-toronto_b_58094c97e4b099c434319388&usg=AOvVaw0v70aurDmwqx3Yh_WFJ8ra'> article </a> highlighting the migration from New York to Toronto. Upon reading the article, one thing that strikes is that people do not just move because of new opportunity, but also factor in the lifestyle. Rising cultural diversity of Toronto has been a magnet for people of culturally-diverse New York. This is something relocation agencies cash upon. They have come up with lots of services that help people explore their future neighborhood. We intend to come up with a smart solution that will provide an impetus to this effort. Our algorithm will factor in varipous lifestyle pointers of a neighborhood and use machine learning to find localities that have matching cultural and lifestyle offerings. This will help find neighborhoods in Toronto, ON that are similar to a given neighborhood in New York.

#### A description of the data and how it will be used to solve the problem.

> The data for 2 cities: Toronto and New York will be used to compare the neighborhoods, and then we rank the neighborhoods from the selected boroughs for the cities.
>> We will use the previous assignments to retrieve the neighborhood and geo co-ordinates for New York and Toronto. We will use Machine Learning (unsupervised learning) to cluster the neighborhoods from the two cities. Data sources are newyor_data.json & the <a href='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'>Toronto Wiki page </a>. Geo locations for Toronto will use this <a href='https://cocl.us/Geospatial_data'>source file</a>. Implementation logic is as follows:

<ul>
    <li>Create a dataset that holds the Geo-cordinates for Toronto's neighborhood</li>
    <li>Use Foursquare APIs to get venues for each of the Toronto's neighborhood</li>
    <li>Sort through the data to identify top 10 common venue categories for each of the Toronto's neighborhood</li>
    <li>Perform the above for New York's neighborhoods</li>
    <li>This step wil be the input. Select a neighborhood in NY for which we are looking for lookalikes in Toronto</li>
    <li>Use K-means on dataset that has all Toronto's neighborhoods plus this neighhborhood</li>
    <li>Then find the cluster which has the NY neighborhood in it and list all Toronto neighborhoods there</li>
</ul>


#### <u>Create a dataset that holds the Geo-cordinates for Toronto's neighborhood</u>

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [8]:
CLIENT_ID = 'IXNL21NRPG4DRXR1KB4S3MBSW4AEGAEGWBTLJMZFPKISSGSE' # your Foursquare ID
CLIENT_SECRET = 'SXM4DBWGGYUUNY4GHMT42GJQKZSENSFRMD3I1FJRCPXFBAS3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
PO_Table = soup.find('table',class_='wikitable sortable')
df = pd.read_html(str(PO_Table))
df_PO = df[0]
df_PO1 = df_PO[df_PO['Borough']!='Not assigned']
df_PO2 = df_PO1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
df_PO2['Neighbourhood'] = df_PO2.apply(
    lambda row: row['Borough'] if (row['Neighbourhood']== 'Not assigned') else row['Neighbourhood'],
    axis=1
)
geo_coord_master = pd.read_csv('https://cocl.us/Geospatial_data')
df_PO3 = pd.merge(df_PO2, geo_coord_master, 
                   left_on='Postcode', right_on = 'Postal Code', how='inner')
df_PO4 = df_PO3[['Postcode','Borough','Neighbourhood','Latitude','Longitude']]
neighborhoods = df_PO4[df_PO4['Borough'].str.contains('Toronto', na=False)][['Borough','Neighbourhood','Latitude','Longitude']]
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


#### <u>Further explore Toronto's neighborhood</u>

In [5]:
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [9]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
Toronto_data = neighborhoods.reset_index(drop=True)

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### <u>Use Foursquare APIs to get venues for each of the Toronto's neighborhood</u>

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [12]:
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()



,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [14]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()

#### <u>Sort through the data to identify top 10 common venue categories for each of the Toronto's neighborhood</u>

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Toronto_neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Toronto_neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Pizza Place,Asian Restaurant,Café,Hotel,Opera House,Gastropub,Concert Hall,Noodle House,Monument / Landmark
1,Berczy Park,Farmers Market,Coffee Shop,Cocktail Bar,Beer Bar,Café,Seafood Restaurant,Jazz Club,Fish Market,Basketball Stadium,Bistro
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Breakfast Spot,Café,Convenience Store,Pet Store,Performing Arts Venue,Italian Restaurant,Intersection,Grocery Store,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Fast Food Restaurant,Park,Comic Shop,Pizza Place,Butcher,Burrito Place,Recording Studio,Restaurant,Brewery
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Coffee Shop,Plane,Sculpture Garden,Boutique,Boat or Ferry


#### <u>Perform the above for New York's neighborhoods</u>
<ul>
    <li>Create a dataset that holds the Geo-cordinates for NY's neighborhood</li>
    <li>Use Foursquare APIs to get venues for each of the NY's neighborhood</li>
    <li>Sort through the data to identify top 10 common venue categories for each of the NY's neighborhood</li>
</ul>

In [17]:
import json # library to handle JSON files
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods_data = newyork_data['features']

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [18]:
address = 'New York, NY'
geolocator = Nominatim(user_agent="NY_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
NY_data = neighborhoods.reset_index(drop=True)
NY_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [19]:
NY_venues = getNearbyVenues(names=NY_data['Neighborhood'],
                                   latitudes=NY_data['Latitude'],
                                   longitudes=NY_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [32]:
# one hot encoding
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot['Neighbourhood'] = NY_venues['Neighbourhood'] 

# move neighborhood column to the first column
cols = list(NY_onehot)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Neighbourhood')))
# use ix to reorder
NY_onehot = NY_onehot.loc[:, cols]

NY_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,...,Warehouse Store,Waste Facility,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
NY_grouped = NY_onehot.groupby('Neighbourhood').mean().reset_index()
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
NY_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
NY_neighborhoods_venues_sorted['Neighbourhood'] = NY_grouped['Neighbourhood']

for ind in np.arange(NY_grouped.shape[0]):
    NY_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

NY_neighborhoods_venues_sorted.head()



,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Supermarket,Spa,Pharmacy,Fast Food Restaurant,Martial Arts Dojo,Gas Station,Breakfast Spot,Spanish Restaurant
1,Annadale,Pizza Place,Park,Sports Bar,Food,Restaurant,Diner,Train Station,Pharmacy,Fast Food Restaurant,Event Service
2,Arden Heights,Pharmacy,Coffee Shop,Bus Stop,Pizza Place,Yoga Studio,Filipino Restaurant,Eye Doctor,Factory,Falafel Restaurant,Farm
3,Arlington,Deli / Bodega,American Restaurant,Boat or Ferry,Grocery Store,Food,Bus Stop,Yoga Studio,Fish Market,Farm,Farmers Market
4,Arrochar,Deli / Bodega,Pizza Place,Bus Stop,Italian Restaurant,Supermarket,Middle Eastern Restaurant,Mediterranean Restaurant,Outdoors & Recreation,Liquor Store,Bagel Shop
